<a href="https://colab.research.google.com/github/nhathoang0110/project2/blob/master/CNN_prj2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import torch  
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import time
import torch.nn.functional as F

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [53]:
!nvidia-smi

Mon Jun 22 09:01:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    34W / 250W |    903MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [54]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from time import time


def load_data(data_path):
  with open(data_path, encoding = 'latin1') as f:
    d_lines = f.read().splitlines()
  data, labels, sentence_lengths = [], [], []
  for line in d_lines:
    features = line.split('<fff>')
    label, doc_id, sentence_len = int(features[0]), int(features[1]), int(features[2])
    vector = [int(ID) for ID in features[3].split()]
    data.append(vector)
    labels.append(label)
    sentence_lengths.append(sentence_len)
  return torch.tensor(data), torch.tensor(labels), torch.tensor(sentence_lengths)
  
train_data, train_labels, train_sentence_lengths = load_data(
    data_path='/content/drive/My Drive/20news-train-encoded.txt'
)
test_data, test_labels, test_sentence_lengths = load_data(
    data_path='/content/drive/My Drive/20news-test-encoded.txt'
)
with open('/content/drive/My Drive/vocab-raw.txt', encoding = 'latin1') as f:
  vocab_size = len(f.read().splitlines())


In [55]:
device = torch.device("cuda")
print(device)

cuda


In [67]:
EMBEDDING_SIZE = 300
MAX_DOC_LENGTH = 500
NUM_CLASSES = 20
device = torch.device("cuda")

batch_size = 50
train_set = TensorDataset(train_data, train_labels)
train_loader = DataLoader(train_set, batch_size = batch_size, shuffle = True)



class CNN(nn.Module):
  def __init__(self, vocab_size, embedding_size, batch_size):
    super(CNN, self).__init__()
    self._vocab_size = vocab_size
    self._embedding_size = embedding_size
    self._batch_size = batch_size
    self.build()

  def build(self):
    self._embedding_layer = nn.Embedding(self._vocab_size+2, self._embedding_size)
    self._convolutional_layer = nn.Sequential(
        nn.Conv2d(1, 500, kernel_size=(5, self._embedding_size)),
        nn.ReLU()
    )
    self._full_connected_layer = nn.Linear(MAX_DOC_LENGTH, NUM_CLASSES)
    self._loss_function = nn.CrossEntropyLoss()
    self._dropout = nn.Dropout(p=0.5)

  def forward(self, inp):
    embeddings = self._embedding_layer(inp).unsqueeze(1)
    outputs = self._convolutional_layer(embeddings).squeeze(3)
    outputs = F.max_pool1d(outputs, kernel_size=outputs.size(2)).squeeze(2)
    outputs = self._dropout((outputs))
    outputs = self._full_connected_layer(outputs)
    return outputs


  # def fit(self, train_loader, max_epochs=50 ,learning_rate=0.01, threshold=1e-3):
  #   # data_set = TensorDataset(train_data, train_labels)
  #   # data_loader = DataLoader(data_set, batch_size = self._batch_size, shuffle = True)
  #   opt = torch.optim.Adam(params = self.parameters(), lr = learning_rate)
  #   self.train()
  #   last_loss = 0
  #   for epoch in range(max_epochs):
  #     new_loss = 0
  #     for data,labels in train_loader:
  #       train = data.to(device)
  #       labels = labels.to(device)
  #       opt.zero_grad()
  #       prediced = self.forward(data)
  #       loss = self._loss_function(prediced, labels)
  #       new_loss += loss
  #       loss.backward()
  #       opt.step()
  #     new_loss = new_loss / len(train_loader)
  #     print('epoch: {}, loss: {}'.format(epoch, new_loss))
  #     print('test accuracy:',self.test(test_loader,device))
  #     if abs(last_loss - new_loss) <= threshold:
  #       return
  #     last_loss=new_loss

  # def test(self, test_loader):
  #   # data_set = TensorDataset(test_data, test_labels)
  #   # data_loader = DataLoader(data_set, batch_size = self._batch_size, shuffle = False)
  #   num_true_predict = 0
  #   self.eval()
  #   for data, labels in test_loader:
  #     data =data.to(device)
  #     labels = labels.to(device)
  #     predicted = torch.argmax(self.forward(data), dim = 1)
  #     num_true_predict += sum((predicted == labels).float())
  #   return num_true_predict*100./len(test_data)


  

In [68]:
from time import time

t=time()
cnn = CNN(
    vocab_size=vocab_size, 
    embedding_size=300,  
    batch_size=32
)


In [69]:

cnn.to(device)

CNN(
  (_embedding_layer): Embedding(18988, 300)
  (_convolutional_layer): Sequential(
    (0): Conv2d(1, 500, kernel_size=(5, 300), stride=(1, 1))
    (1): ReLU()
  )
  (_full_connected_layer): Linear(in_features=500, out_features=20, bias=True)
  (_loss_function): CrossEntropyLoss()
  (_dropout): Dropout(p=0.5, inplace=False)
)

In [78]:
opt = torch.optim.Adam(cnn.parameters(), lr = learning_rate)

In [85]:
learning_rate=0.001
max_epochs=50
error = nn.CrossEntropyLoss()
def test(model, test_data, test_labels):
  data_set = TensorDataset(test_data, test_labels)
  data_loader = DataLoader(data_set, batch_size = 32, shuffle = False)
  num_true_predict = 0
  model.eval()
  for data, labels in data_loader:
    data =data.to(device)
    labels = labels.to(device)
    predicted = torch.argmax(model(data), dim = 1)
    num_true_predict += sum((predicted == labels).float())
  return num_true_predict*100./len(test_data)
cnn.train()
last_loss = 0
for epoch in range(max_epochs):
  new_loss = 0
  for data,labels in train_loader:
    train = data.to(device)
    labels = labels.to(device)
    opt.zero_grad()
    prediced = cnn(train)
    loss = error(prediced, labels)
    new_loss += loss
    loss.backward()
    opt.step()
  new_loss = new_loss / len(train_loader)
  print('epoch: {}, loss: {}'.format(epoch, new_loss))
  print('test accuracy:',test(cnn,test_data, test_labels))
  print('train accuracy:', test(cnn,train_data,train_labels))
  # if abs(last_loss - new_loss) <= threshold:
  #   return
  last_loss=new_loss




epoch: 0, loss: 29.51390266418457
test accuracy: tensor(71.6675, device='cuda:0')
train accuracy: tensor(96.5088, device='cuda:0')
epoch: 1, loss: 2.1171176433563232
test accuracy: tensor(72.5305, device='cuda:0')
train accuracy: tensor(97.3838, device='cuda:0')
epoch: 2, loss: 0.5601410269737244
test accuracy: tensor(72.8624, device='cuda:0')
train accuracy: tensor(97.6136, device='cuda:0')
epoch: 3, loss: 0.5515150427818298
test accuracy: tensor(73.3271, device='cuda:0')
train accuracy: tensor(97.7373, device='cuda:0')
epoch: 4, loss: 0.1460321545600891
test accuracy: tensor(73.3404, device='cuda:0')
train accuracy: tensor(97.7550, device='cuda:0')
epoch: 5, loss: 0.10092546045780182
test accuracy: tensor(73.3404, device='cuda:0')
train accuracy: tensor(97.7727, device='cuda:0')
epoch: 6, loss: 0.08729109168052673
test accuracy: tensor(73.3537, device='cuda:0')
train accuracy: tensor(97.7815, device='cuda:0')
epoch: 7, loss: 0.08204309642314911
test accuracy: tensor(73.3802, device='

KeyboardInterrupt: ignored

In [65]:
print('training time:',time()-t,'s')


RuntimeError: ignored